# Travel Time 

### Goal of the notebook

The purpose of this Jupyter notebook is to request the travel time in second from the origin to destination using google API.
***
**Outputs:** 

- travel_time


**Inputs:** 

- ori_centroid_id
- des_centroid_id
- select_time

External centroid data:

- External_centroid_googlemap.xlsx




**Dependent scripts:** 
None

 **IMPORTANT:** Run the iPython cell below in order to import the necessary packages.

In [119]:
import numpy as np
import scipy
import matplotlib
import pandas as pd
import numpy as np
import json as simplejson
import urllib
from urllib import request
import requests, json 
from bs4 import BeautifulSoup
from datetime import datetime

### Load external centroid data

This function loads the external centroid data with longtitue and latitute from dropbox into python notebook

In [120]:
# Root path of Fremont Dropbox
import os
import sys
# We let this notebook to know where to look for fremontdropbox module
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from fremontdropbox import get_dropbox_location

path_dropbox = get_dropbox_location()

path_data=os.path.join(path_dropbox, "Data Collection/Data processing/Data in process/Demand/External_centroid_googlemap.xlsx")

external_data=pd.read_excel(path_data)
external_data=external_data.set_index('External_Centroid_ID')
external_data

,NB/EB,SB/WB
External_Centroid_ID,,
4,"37.512253,-121.948685","37.512406,-121.948839"
5,"37.533005,-121.954678","37.533122,-121.954726"
6,"37.542265,-121.949716","37.542124,-121.949697"
7,"37.546844,-121.946638","37.546813,-121.946663"
9,"37.544235,-121.947014","37.544211,-121.947038"
11,"37.546699,-121.944992","37.546688,-121.945014"
12,"37.551385,-121.942218","37.551346,-121.942392"
13,"37.547740,-121.919675","37.547653,-121.919938"
14,"37.539300,-121.914089","37.539358,-121.914051"


### Google API key

This cell is to define the Google API key that we will be using

In [121]:
#This is my own google API key which does not work for now
Google_API_KEY=

### Function to request Travel Time

This function is defined to request the travel time using Google API

In [122]:
def GetTravelTime(ori_external_id, des_external_id, time):
    orig_coord_NB_EB = external_data.loc[ori_external_id,'NB/EB']
    orig_coord_SB_WB = external_data.loc[ori_external_id,'SB/WB']
    dest_coord_NB_EB = external_data.loc[des_external_id,'NB/EB']
    dest_coord_SB_WB = external_data.loc[des_external_id,'SB/WB']
    l=[]
    url ='https://maps.googleapis.com/maps/api/distancematrix/json?'
    
    r = url + "origins=" + orig_coord_NB_EB + "&destinations=" + dest_coord_NB_EB + \
         "&mode=driving&traffic_model=best_guess&departure_time="+time+"&language=en-EN&sensor=false&key=" \
         + Google_API_KEY
    result= simplejson.load(urllib.request.urlopen(r))
    driving_time1 = result['rows'][0]['elements'][0]['duration_in_traffic']['value']
    l.append(driving_time1)
    
    r = url + "origins=" + orig_coord_NB_EB + "&destinations=" + dest_coord_SB_WB + \
         "&mode=driving&traffic_model=best_guess&departure_time="+time+"&language=en-EN&sensor=false&key=" \
         + Google_API_KEY
    result= simplejson.load(urllib.request.urlopen(r))
    print(result)
    driving_time2 = result['rows'][0]['elements'][0]['duration_in_traffic']['value']
    l.append(driving_time2)
    
    r = url + "origins=" + orig_coord_SB_WB + "&destinations=" + dest_coord_NB_EB + \
         "&mode=driving&traffic_model=best_guess&departure_time="+time+"&language=en-EN&sensor=false&key=" \
         + Google_API_KEY
    result= simplejson.load(urllib.request.urlopen(r))
    driving_time3 = result['rows'][0]['elements'][0]['duration_in_traffic']['value']
    l.append(driving_time3)
    
    r = url + "origins=" + orig_coord_SB_WB + "&destinations=" + dest_coord_SB_WB + \
        "&mode=driving&traffic_model=best_guess&departure_time="+time+"&language=en-EN&sensor=false&key=" \
         + Google_API_KEY 
    result= simplejson.load(urllib.request.urlopen(r))
    driving_time4 = result['rows'][0]['elements'][0]['duration_in_traffic']['value']
    l.append(driving_time4)
    
    return min(l)


### Function to show the travel time

This function is call the above function with parameter and show the corresponding travel time

In [123]:
GetTravelTime(5,6,'now')

{'destination_addresses': ['1900 Gomes Rd, Fremont, CA 94539, USA'], 'origin_addresses': ['3623 Washington Blvd, Fremont, CA 94538, USA'], 'rows': [{'elements': [{'distance': {'text': '1.4 km', 'value': 1423}, 'duration': {'text': '3 mins', 'value': 186}, 'duration_in_traffic': {'text': '4 mins', 'value': 210}, 'status': 'OK'}]}], 'status': 'OK'}


182